In [1]:
import os

In [2]:
%pwd

'd:\\voting_application\\Text_summariser-NLP-\\Text_summariser-NLP-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\voting_application\\Text_summariser-NLP-\\Text_summariser-NLP-'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path: Path
    tokenizer_name: Path

In [8]:
from text_summarizer.constants  import *
from text_summarizer.utlis.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_tranformation_config= DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name =config.tokenizer_name
        )
        return data_tranformation_config

In [12]:
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-07-01 23:33:12,993: INFO: config: PyTorch version 2.2.1 available.]


In [29]:
class DataTransformation:
    def __init__(
            self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    
    def convert_examples_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length =1024 , truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length =1024, truncation = True)

        return{
        'input_ids' : input_encoding['input_ids'],
        'attention_mask' : input_encoding['attention_mask'],
        'labels' : target_encoding['input_ids']}

    def converter(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
    

In [24]:
pip install protobuf

   ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
   ---------------------------- ----------- 307.2/426.9 kB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 426.9/426.9 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
try:
    config = ConfigurationManager()
    data_tranformation_config = config.get_data_transformation_config()
    data_tranformation = DataTransformation(config= data_tranformation_config)
    data_tranformation.converter()

except Exception as e:
    raise e

[2024-07-01 23:48:49,540: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-01 23:48:49,548: INFO: common: yaml file: paramas.yaml loaded successfully]
[2024-07-01 23:48:49,556: INFO: common: created directory at: artifacts]
[2024-07-01 23:48:49,556: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\dwive\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 19639.83 examples/s]
